In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
pd.options.display.max_columns = 90
import copy
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/marketing-strategy-personalised-offer/sample.csv
/kaggle/input/marketing-strategy-personalised-offer/train_data.csv
/kaggle/input/marketing-strategy-personalised-offer/test_data.csv


In [2]:
data  = pd.read_csv('/kaggle/input/marketing-strategy-personalised-offer/train_data.csv')
eval_data = pd.read_csv('/kaggle/input/marketing-strategy-personalised-offer/test_data.csv')
data.nunique()

offer expiration                         2
income_range                             9
no_visited_Cold drinks                   5
travelled_more_than_15mins_for_offer     2
Restaur_spend_less_than20                5
Marital Status                           5
restaurant type                          5
age                                      8
Prefer western over chinese              2
travelled_more_than_25mins_for_offer     2
travelled_more_than_5mins_for_offer      1
no_visited_bars                          5
gender                                   2
car                                      5
restuarant_same_direction_house          2
Cooks regularly                          2
Customer type                            4
Qualification                            6
is foodie                                2
no_Take-aways                            5
Job/Job Industry                        25
restuarant_opposite_direction_house      2
has Children                             2
visit resta

### Convert days to Hours

In [3]:
data  = pd.read_csv('/kaggle/input/marketing-strategy-personalised-offer/train_data.csv')
eval_data = pd.read_csv('/kaggle/input/marketing-strategy-personalised-offer/test_data.csv')
def convert_to_hours(row):
    if "hours" in row['offer expiration']:
#         row['offer expiration'] = int(row['offer expiration'][:-5])
        row['offer expiration'] = 0
    elif "days" in row['offer expiration']:
#         row['offer expiration'] = int(row['offer expiration'][:-4])*24
        row['offer expiration'] = 1
    return row
def save_data():
    global data, eval_data
    data.to_parquet('data.parquet')
    eval_data.to_parquet('eval_data.parquet')

data = data.apply(convert_to_hours,axis=1)
eval_data = eval_data.apply(convert_to_hours,axis=1)
def pythonise_col_names(df: pd.DataFrame):
    col_names = list(df.columns)
    rename_stubs = {}
    for x in col_names:
        if x == 'Offer Accepted':
            r = 'target'
            rename_stubs[x]=r
        elif "/" in x:
            r = x.split('/')[1]
            r = r.replace(' ','_').lower()
            rename_stubs[x]=r
        elif "-" in x:
            r = x.replace('-','_').lower()
            r = r.replace(' ','_').lower()
            rename_stubs[x]=r
        elif " " in x:
            r = x.replace(' ','_').lower()
            rename_stubs[x]=r
        else:
            r = x.lower()
            rename_stubs[x] = r 
    return rename_stubs
data.rename(pythonise_col_names(data), inplace=True,axis=1)
eval_data.rename(pythonise_col_names(eval_data), inplace=True,axis=1)
gender_code = {
    'Male':0,
    'Female':1
}
def binarize_gender(row):
    global gender_code
    row['gender'] = gender_code[row['gender']]
    return row
data = data.apply(binarize_gender,axis=1)
eval_data = eval_data.apply(binarize_gender,axis=1)


ages_ord = ['below21','21', '26', '31', '36', '41', '46', '50plus']
ordinal_age_codes =  {x: idx for idx, x in enumerate(ages_ord)}
income_codes = { 
    'Less than ₹12500':1,
    '₹12500 - ₹24999' :2,
    '₹25000 - ₹37499' :3,
    '₹37500 - ₹49999' :4,
    '₹50000 - ₹62499' :5,
    '₹62500 - ₹74999' :6,
    '₹75000 - ₹87499' :7,
    '₹87500 - ₹99999' :8,
    '₹100000 or More' :9,
}

data['ordinal__income_range'] = 0
eval_data['ordinal__income_range'] = 0
data['ordinal__age'] = 0
eval_data['ordinal__age'] = 0

def codify_special_ordinal_range(row):
    global income_codes, ordinal_age_codes
    row['ordinal__income_range'] = income_codes[row['income_range']]
    row['ordinal__age'] = ordinal_age_codes[row['age']]
    return row
data = data.apply(codify_special_ordinal_range,axis=1)
eval_data = eval_data.apply(codify_special_ordinal_range,axis=1)

ordinal_category_codes = {
    '4~8': 3,
    'less1': 1,
    'never': 0,
    '1~3': 2,
    'gt8': 4,
    np.nan: np.nan}

# data = data.apply(codify_cold_drink_consumption,axis=1)
# eval_data = eval_data.apply(codify_cold_drink_consumption,axis=1)
ord_cols = ["restaur_spend_less_than20","no_take_aways","restaur_spend_greater_than20","no_visited_bars","no_visited_cold_drinks"]
col_ord_names = ["ordinal__" + x for x in ord_cols]
data[col_ord_names] = 0
eval_data[col_ord_names] = 0
def codify_ordinal_columns(row):
    global col_ord_names, ordinal_category_codes, ord_cols
    for ord_col,col_ord_name in zip(ord_cols,col_ord_names):
        try:
            row[col_ord_name] = int(ordinal_category_codes[row[ord_col]])
        except ValueError:
            row[col_ord_name] = ordinal_category_codes[row[ord_col]]
    return row
data = data.apply(codify_ordinal_columns,axis=1)
eval_data = eval_data.apply(codify_ordinal_columns,axis=1)

"""
Car Feature:
Label NaN as unknown for now
"""
data.car.fillna("unknown",inplace=True)
eval_data.car.fillna("unknown",inplace=True)

nominal_cs = ['restaurant_type','marital_status','climate',"drop_location","job_industry","customer_type","car","qualification"]
nominal_col_names = ["nominal__" + x for x in nominal_cs]
master_nominals= pd.concat([data[nominal_cs],eval_data[nominal_cs]], ignore_index=True,axis=0)
nominal_encoder = {nc:{} for nc in nominal_cs}
for c in nominal_cs:
    unique_vals = list(master_nominals[c].unique())
    nominal_codes = {x: idx for idx, x in enumerate(unique_vals)}
    nominal_encoder[c] = nominal_codes
data[nominal_col_names] = 0
eval_data[nominal_col_names] = 0
def codify_nominal_columns(row):
    global nominal_col_names, nominal_encoder, nominal_cs
    for nom_col,col_nom_name in zip(nominal_cs,nominal_col_names):
        try:
            row[col_nom_name] = int(nominal_encoder[nom_col][row[nom_col]])
        except ValueError:
            row[col_nom_name] = nominal_encoder[nom_col][row[nom_col]]
    return row
data = data.apply(codify_nominal_columns,axis=1)
eval_data = eval_data.apply(codify_nominal_columns,axis=1)

def drop_const_and_enc_columns(df):
    cols_to_drop = [
        'restaur_spend_less_than20',
        'no_take_aways',
        'restaur_spend_greater_than20',
        'no_visited_bars',
        'no_visited_cold_drinks',
        'restaurant_type',
        'marital_status',
        'climate',
        'qualification',
        'drop_location',
        'job_industry',
        'customer_type',
        'car',
        'income_range',
        "travelled_more_than_5mins_for_offer",
        'age'
    ]
    df = df.drop(cols_to_drop,axis=1)
    return df
data = drop_const_and_enc_columns(data)
eval_data = drop_const_and_enc_columns(eval_data)

data_original_col_order = [
    'offer_expiration',
    'ordinal__income_range',
    'ordinal__no_visited_cold_drinks',
    'travelled_more_than_15mins_for_offer',
    'ordinal__restaur_spend_less_than20',
    'nominal__marital_status',
    'nominal__restaurant_type',
    'ordinal__age',
    'prefer_western_over_chinese',
    'travelled_more_than_25mins_for_offer',
    'ordinal__no_visited_bars',
    'gender',
    'nominal__car',
    'restuarant_same_direction_house',
    'cooks_regularly',
    'nominal__customer_type',
    'nominal__qualification',
    'is_foodie',
    'ordinal__no_take_aways',
    'nominal__job_industry',
    'restuarant_opposite_direction_house',
    'has_children',
    'visit_restaurant_with_rating_(avg)',
    'temperature',
    'ordinal__restaur_spend_greater_than20',
    'travel_time',
    'nominal__climate',
    'nominal__drop_location',
    'prefer_home_food',
    'target'
]
eval_data_original_col_order = copy.deepcopy(data_original_col_order)
eval_data_original_col_order.remove('target')

data = data.loc[:,data_original_col_order]
eval_data = eval_data.loc[:,eval_data_original_col_order]
binary_features= [
    'offer_expiration',
    'travelled_more_than_15mins_for_offer',
    'prefer_western_over_chinese',
    'travelled_more_than_25mins_for_offer',
    'restuarant_same_direction_house',
    'cooks_regularly',
    'is_foodie',
    'restuarant_opposite_direction_house',
    'has_children',
    'prefer_home_food',
    'gender'
]
b_f_rn = {x:"binary__"+ x for x in binary_features}
data.rename(b_f_rn,axis=1,inplace=True)
eval_data.rename(b_f_rn,axis=1,inplace=True)
remaining_ordinals = {
    "visit_restaurant_with_rating_(avg)": "ordinal__rest_rating",
    "temperature": 'ordinal__temperature',
    "travel_time": "ordinal__travel_time"
}
def rename_remaining_ord(df):
    df = df.rename(remaining_ordinals,axis=1)
    return df
data = rename_remaining_ord(data)
eval_data = rename_remaining_ord(eval_data)

target_codes = {"Yes":1,"No":0}
def binarize_target(row):
    global target_codes
    row['target'] = target_codes[row['target']]
    return row
data = data.apply(binarize_target,axis=1)
save_data()